In [4]:
#@title Run this to prepare our libraries and data! { display-mode: "form" } 

# networking
!pip -q install pyngrok
!pip -q install streamlit
!pip -q install patool

import cv2
import gdown
import glob
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import patoolib

from joblib import dump
from pyngrok import ngrok
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

DATA_ROOT = '/content'
os.makedirs(DATA_ROOT, exist_ok=True)
max_samples = 3000

blood_slide_url = 'https://drive.google.com/uc?id=1lffxAG8gykh1dh1pCP34uRkH3XMwuNt-'
blood_slide_path = os.path.join(DATA_ROOT, 'blood_slide.jpg')
gdown.download(blood_slide_url, blood_slide_path, True)

!wget "https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/malaria_images.zip"
!unzip "malaria_images.zip"
malaria_imgs_path = os.path.join(DATA_ROOT, 'malaria_images.zip')

print("Downloaded Data")

u_malaria_img_paths = glob.glob('/content/malaria_images/Uninfected/*png')
p_malaria_img_paths = glob.glob('/content/malaria_images/Parasitized/*png')

NUM_SAMPLES = len(u_malaria_img_paths) + len(p_malaria_img_paths)

X = []
y = []

X_g = []

for i in tqdm(range(max_samples)):
  img = cv2.imread(u_malaria_img_paths[i])
  X.append(cv2.resize(img,(50,50)))

  gray_img = cv2.imread(u_malaria_img_paths[i],0)
  X_g.append(cv2.resize(gray_img,(50,50)))

  y.append(0)

for i in tqdm(range(max_samples)):
  img = cv2.imread(p_malaria_img_paths[i])
  X.append(cv2.resize(img,(50,50)))

  gray_img = cv2.imread(p_malaria_img_paths[i],0)
  X_g.append(cv2.resize(gray_img,(50,50)))

  y.append(1)

X = np.stack(X)
X_g = np.stack(X_g)
X_reshaped = np.reshape(X_g,(X_g.shape[0],2500))

y = np.array(y)

blood_samples_dir = 'blood_samples'
if (os.path.exists(blood_samples_dir) == False):
  os.mkdir(blood_samples_dir)

for i, img in enumerate(X[2995:3005]):
  plt.imsave('test_img_{}.jpg'.format(i), img)
  
print("Created our X and y variables")


--2022-04-17 00:15:48--  https://storage.googleapis.com/inspirit-ai-data-bucket-1/Data/Deep%20Dives/malaria_images.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 173.194.216.128, 173.194.217.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 353755287 (337M) [application/zip]
Saving to: ‘malaria_images.zip.4’

malaria_images.zip. 100%[===================>] 337.37M   222MB/s    in 1.5s    

2022-04-17 00:15:50 (222 MB/s) - ‘malaria_images.zip.4’ saved [353755287/353755287]

Archive:  malaria_images.zip
replace malaria_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_162.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: Downloaded Data


100%|██████████| 3000/3000 [00:03<00:00, 986.81it/s]

Created our X and y variables


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y, test_size = 0.33)
model = SVC()
model.fit(X_train, y_train)
print(model.score(X_test, y_test))

0.6984848484848485


In [9]:
dump(model, "model.joblib")

['model.joblib']

In [51]:
%%writefile app.py
import streamlit as st
from joblib import load #reads the pre-isting model file
import numpy as np
import cv2

st.title('Malaria Classification') #Change the title here!

st.image("""...""")

with st.container():
  st.subheader("First lets understand what Malaria is")
  st.caption(""" Malaria is an intermittent and remittent fever caused by a protozoan parasite that invades the red blood cells. 
  The parasite is transmitted by mosquitoes in many tropical and subtropical regions.""")
  st.subheader("Now lets see what we can do for you")
  st.caption(""" You can upload a file containing a scan of your red blood cells and we can quickly quickly classify it as malaria infected or not """)
    
uploaded_file = st.file_uploader("Upload image for classification")
  
model = load("model.joblib")


if uploaded_file is not None:    
    file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1) 
    st.image(image, caption="Input Image", channels="BGR")
    small = cv2.resize(image, (50,50)) #YOUR CODE HERE: specify image, dimensions
    gray = cv2.cvtColor(small, cv2.COLOR_BGR2GRAY)
    st.image(gray, caption="Gray Image")
    flattened = np.reshape(gray, (1, 2500))
    prediction = model.predict(flattened)
    if(prediction[0] == 1):
      st.write("You have Malaria.")
      with st.expander("Do you want to learn more about the necessary procedures to take?"):
        st.write("""
          Here are some helpfull links to seek medical attention
          and what procedure to aquire.

          https://www.cdc.gov/malaria/about/faqs.html
          https://www.cdc.gov/malaria/diagnosis_treatment/clinicians1.html 
          
        """)
    else:
      st.write("You don't have Malaria.")
      

Overwriting app.py


In [52]:
!ngrok authtoken 26IDZr3qAIq1POCz9X3UFYyBXcy_duie66fnSoPsELaPMcrR

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

NgrokTunnel: "http://f1f9-104-196-0-75.ngrok.io" -> "http://localhost:80"
2022-04-17 02:15:53.854 INFO    numexpr.utils: NumExpr defaulting to 2 threads.
